In [176]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import json
import os
from tqdm import tqdm
from collections import Counter
import numpy as np
import csv
import string
import re

In [177]:
MAIN_PATH = "/home/jack/datasets/polish_rap/"
DICTIONARY_PATH = "/home/jack/datasets/polish_dictionaries/slownik_morfosyntaktyczny.csv"

In [178]:
with open(DICTIONARY_PATH, "r") as f:
    slownik_pl = pd.read_csv(f, sep=";", header=None)

In [180]:
slownik_pl.drop(2, axis=1, inplace=True)

ValueError: labels [2] not contained in axis

In [181]:
slownik_pl.head(10)

,0,1
0,AA,AA
1,AAP,AAP
2,ABB,ABB
3,ABBA,ABB
4,ABBA,ABB-ach
5,ABBA,ABB-ami
6,ABBA,ABB-ie
7,ABBA,ABB-o
8,ABBA,ABB-om
9,ABBA,ABB-y


In [182]:
slownik_pl[slownik_pl[1] == "ul"][0].to_string(index=False).split()[0]

'ul'

In [183]:
def lemmatize_df(word):
    t = slownik_pl[slownik_pl[1] == word]
    return t[0].to_string(index=False).split()[0] if not t.empty else word

In [184]:
# %%timeit
lemmatize_df("poszukiwani")

'poszukiwany'

In [185]:
slownik_pl_dict = slownik_pl.set_index(1)[0].to_dict()
slownik_pl_dict = {normalize(k): normalize(v) for k, v in slownik_pl_dict.items()}
len(slownik_pl_dict)

4622075

In [172]:
with open("/home/jack/datasets/polish_dictionaries/slownik_znormalizowany.json", "w") as f:
    json.dump(slownik_pl_dict, f)


In [186]:
def lemmatize(word):
    return slownik_pl_dict[word] if word in slownik_pl_dict else word

In [187]:
def normalize(word):
    word = word.lower()
    exclude = set(string.punctuation)
    regex = re.compile('[%s]' % re.escape(string.punctuation))

    return regex.sub('', word)

In [188]:
len(slownik_pl)

4811854

In [189]:
# %%timeit
lemmatize("poszukiwani")

'poszukiwać'

In [190]:
def optionally_mkdir(path):
    if not os.path.exists(path):
        os.mkdir(path)

In [191]:
def lemmatize_all():
    rappers = os.listdir(MAIN_PATH + "ok_lyrics/")
    optionally_mkdir(MAIN_PATH + "stm_lyrics/")

    for rapper in tqdm(rappers):
        songs = os.listdir(MAIN_PATH + "ok_lyrics/" + rapper)
        optionally_mkdir(MAIN_PATH + "stm_lyrics/" + rapper)
        for song in songs:
            new_song = ""
            if not os.path.exists(MAIN_PATH + "stm_lyrics/" + rapper + "/" + song):
                with open(MAIN_PATH + "ok_lyrics/" + rapper + "/" + song, "r") as f:
                    old_song_json = json.load(f)
                old_song = old_song_json["lyrics"]
                for line in old_song.split("\n"):
                    for word in line.split(" "):
                        word = normalize(word)
                        new_word = lemmatize_dict(word)
                        new_song += new_word
                        new_song += " "
                    new_song += "\n"
                #warning - it leaves space at the end of each line and newline at the end of file
                new_song_json = old_song_json
                new_song_json["lyrics"] = new_song
                with open(MAIN_PATH + "stm_lyrics/" + rapper + "/" + song, "w") as f:
                    old_song = json.dump(new_song_json, f)

In [192]:
# %%timeit
lemmatize_all()

In [193]:
def lemmatize_strongs():
    new_strongs = set()
    with open(MAIN_PATH + "strong_language.json", "r") as f:
        strongs = json.load(f)
    for strong in strongs:
        new_strongs.add(lemmatize(strong))
    new_strongs = list(new_strongs)

    with open(MAIN_PATH + "strong_language_lem.json", "w") as f:
        json.dump(new_strongs, f)
    

In [194]:
lemmatize_strongs()